In [ ]:
import polars as pl
from polars import col
from bill_law_similarity.vector import preprocess, compute_cosine_similarity
from bill_law_similarity.docu_toads import docu_toads_similarity
from bill_law_similarity.bow import bow_overlap, bow_pos_overlap
from bill_law_similarity.string_diff import diff
from bill_law_similarity.deep import BertSimilarity
from bill_law_similarity.string_diff import sw_align
import jaro

%load_ext autoreload
%autoreload 2

In [ ]:
def load_df(type, lang="de", preprocessing=True):
    df = pl.read_csv(f"data/annotated/{type}_sub_sec_pairs.csv").rename(
        {"sec_a_text": "bill_text", "sec_b_text": "law_text"}
    )
    if not preprocessing:
        return df
    preprocessed = preprocess(df, lang)
    return preprocessed


In [ ]:
# train_metrics_en, val_metrics_en, test_metrics_en = pipeline("en")
train_metrics_en = pl.read_csv("data/results/train_metrics_en.csv")
val_metrics_en = pl.read_csv("data/results/val_metrics_en.csv")
test_metrics_en = pl.read_csv("data/results/test_metrics_en.csv")

In [ ]:
test_de = load_df("test", "de", preprocessing=False).with_columns([
    pl.col("label") / 4
])
test_en = load_df("test", "en", preprocessing=False).with_columns([
    pl.col("label") / 4
])

# Hard Sentences

In [ ]:
import itertools
a = ["We prefer environmental protection over economic growth", "We prefer economic growth over environmental protection", "We prefer climate change mitigation over economic performance", "We prefer economic performance over climate change mitigation"]
# combine every two sentences
b = list(itertools.combinations(a, 2))

In [ ]:
example = pl.from_records(b, orient="row", schema=["bill_text", "law_text"])
example_pre = preprocess(example, "en")
pl.concat([example,similarity_metrics(example_pre)[:,10:]], how="horizontal")#.write_csv("christian_beispiel.csv")